<a href="https://colab.research.google.com/github/Dadoyen2/Geometry-Optimization/blob/main/geometry_optimization_in_progress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
import requests
import math
def get_coordinate(url):
    atoms = []
    response = requests.get(url)
    response.raise_for_status()
    lines = response.text.splitlines()

    # Start parsing for atom data directly after identifying the pattern
    for line in lines:
        parts = line.split()
        # Check if line contains a potential atom data pattern
        if len(parts) >= 4 and parts[3].isalpha():
            try:
                # Attempt to parse the first three values as coordinates
                x, y, z = map(float, parts[:3])
                atom_type = parts[3]
                atoms.append((atom_type, x, y, z))
            except ValueError:
                # Skip lines that don't fit the pattern (ensures generality)
                continue

    return atoms

def print_atom_coordinates(atoms):
    print(f"The input file has: {len(atoms)} atoms")
    print("Atoms and coordinates (in Å):")
    for atom in atoms:
        print(f"{atom[0]:<2} {atom[1]:>8.4f} {atom[2]:>8.4f} {atom[3]:>8.4f}")

# Example URL to use with the general extraction function
url = 'https://raw.githubusercontent.com/Dadoyen2/Geometry-Optimization/main/ethane.mol2'
atoms = get_coordinate(url)
print_atom_coordinates(atoms)




The input file has: 8 atoms
Atoms and coordinates (in Å):
C   -0.7560   0.0500   0.0000
C    0.7560   0.0000   0.0500
H   -1.1404   0.6586   0.7845
H   -1.1404   0.3501  -0.9626
H   -1.1405  -1.0087   0.4781
H    1.1404  -0.3501   0.9626
H    1.1405   1.0087  -0.1781
H    1.1404  -0.6586  -0.7845


In [110]:
import numpy as np
import requests

def bond_length(coord1, coord2):
    return np.linalg.norm(np.array(coord2) - np.array(coord1))

def bond_angle(coord1, coord2, coord3):
    vec1 = np.array(coord1) - np.array(coord2)
    vec2 = np.array(coord3) - np.array(coord2)
    cos_theta = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    return np.degrees(np.arccos(cos_theta))



def dihedral_angle(coord1, coord2, coord3, coord4):
    # Define the vectors
    vec1 = np.array(coord1) - np.array(coord2)  # r_AB
    vec2 = np.array(coord3) - np.array(coord2)  # r_BC
    vec3 = np.array(coord4) - np.array(coord3)  # r_CD

    # Calculate normal vectors to planes
    t = np.cross(vec1, vec2)  # Normal to plane ABC
    u = np.cross(vec2, vec3)  # Normal to plane BCD

    # Calculate vector perpendicular to both normal vectors
    v = np.cross(t, u)

    # Compute cos(phi)
    cos_phi = np.dot(t, u) / (np.linalg.norm(t) * np.linalg.norm(u))

    # Compute sin(phi)
    sin_phi = np.dot(vec2 / np.linalg.norm(vec2), v) / (np.linalg.norm(t) * np.linalg.norm(u))

    # Correct numerical inaccuracies (clipping for cos_phi)
    cos_phi = np.clip(cos_phi, -1.0, 1.0)

    # Calculate dihedral angle
    angle_rad = np.arctan2(sin_phi, cos_phi)  # Angle in radians
    angle_degree = np.degrees(angle_rad)     # Convert radians to degrees

    # Normalize angle to the range 0° to 360°
    if angle_degree < 0:
        angle_degree += 360
    angle_rad = np.radians(angle_degree)

  #  return angle_degree, angle_rad

    # Subtract the angle from 180 degrees
    corrected_angle_degree = angle_degree -180
    corrected_angle_rad = np.radians(corrected_angle_degree)

    return corrected_angle_degree, corrected_angle_rad


# Calculate potential energy and count parameters
def calculate_energy(atoms, bond_params, angle_params, dihedral_params, lj_params):
    bond_count, angle_count, dihedral_count = 0, 0, 0
    stretch_energy, bend_energy, torsion_energy, vdw_energy = 0.0, 0.0, 0.0, 0.0

# Stretching (Bond) Energy Calculation
    # Parameters for bond lengths and constants
bond_params = {
    ('C', 'H'): (350, 1.11),  # Example: k_b and r_0 for C-H bond
    ('C', 'C'): (300, 1.53),  # Example: k_b and r_0 for C-C bond
    #
}

# Define distance cutoffs to distinguish bonded vs non-bonded interactions
bond_cutoffs = {
    ('C', 'H'): (0.5, 1.5),  # Minimum and maximum cutoff for C-H bond detection
    ('C', 'C'): (1.0, 2.0),  # Minimum and maximum cutoff for C-C bond detection
    # Add other cutoffs as needed
}

# Stretching (Bond) Energy Calculation with flexibility for any C-H or hydrocarbon bonds
stretch_energy = 0.0
bond_count = 0
visited_bonds = set()  # To keep track of counted bonds
bonds_new = []
for i in range(len(atoms) - 1):
    for j in range(i + 1, len(atoms)):
        bond_type = tuple(sorted((atoms[i][0], atoms[j][0])))  # Determine bond type dynamically

        # Only consider bond types defined in bond_params and bond_cutoffs
        if bond_type in bond_params and bond_type in bond_cutoffs:
            # Calculate bond length
            r = bond_length(atoms[i][1:], atoms[j][1:])

            # Apply distance cutoff to avoid counting non-bonded pairs
            min_cutoff, max_cutoff = bond_cutoffs[bond_type]
            if min_cutoff <= r <= max_cutoff and (i, j) not in visited_bonds:
                visited_bonds.add((i, j))  # Mark this bond as visited

                # Get the bond parameters
                k_b, r_0 = bond_params[bond_type]
                bonds_new.append([i,j])
                # Calculate energy for this bond and add to total
                bond_energy = k_b * (r - r_0) ** 2
                stretch_energy += bond_energy
                bond_count += 1

                # Debugging output for verification
                print(f"Bond {bond_type[0]}-{bond_type[1]}: Length = {r:.2f} Å, Energy = {bond_energy:.4f} kcal/mol")

# Output the final results
print(f"\nTotal Stretching (Bond) Energy: {stretch_energy:.4f} kcal/mol")
print(f"Number of Bonds: {bond_count}")




Bond C-C: Length = 1.51 Å, Energy = 0.0802 kcal/mol
Bond C-H: Length = 1.06 Å, Energy = 0.7181 kcal/mol
Bond C-H: Length = 1.08 Å, Energy = 0.3345 kcal/mol
Bond C-H: Length = 1.22 Å, Energy = 4.5190 kcal/mol
Bond C-H: Length = 1.05 Å, Energy = 1.2466 kcal/mol
Bond C-H: Length = 1.10 Å, Energy = 0.0156 kcal/mol
Bond C-H: Length = 1.13 Å, Energy = 0.1463 kcal/mol

Total Stretching (Bond) Energy: 7.0602 kcal/mol
Number of Bonds: 7


In [111]:

#bending energy calculayion
bond_distance_threshold = 1.6  # Bonding threshold in angstroms
equilibrium_angle_degrees = 109.5  # Equilibrium bond angle for sp³ carbons


# Define k_a values for bending energy
ka_values_degree = {
    ('H', 'C', 'H'): 35,
    ('H', 'C', 'C'): 35,
    ('C', 'C', 'C'): 60
}

# Initialize bending energy calculations
bend_energy_degrees = 0.0
bend_energy_radians = 0.0
angle_count = 0
angles_new= []
# Example atoms

# Calculate angles and bending energy
for j in range(len(atoms)):
    if atoms[j][0] == 'C':  # Central atom must be carbon
        # Find atoms bonded to the central carbon atom
        bonded_atoms = [
            i for i in range(len(atoms))
            if i != j and bond_length(atoms[i][1:], atoms[j][1:]) <= bond_distance_threshold
        ]

        # Iterate over unique pairs of bonded atoms
        for m in range(len(bonded_atoms)):
            for n in range(m + 1, len(bonded_atoms)):
                i, k = bonded_atoms[m], bonded_atoms[n]

                # Determine the angle type and corresponding k_a value
                angle_type = (atoms[i][0], atoms[j][0], atoms[k][0])
                ka_degree = ka_values_degree.get(angle_type, 35)  # Default to 35 if not found
                angles_new.append([i,j,k])
                # Calculate bond angle in degrees
                angle_degrees = bond_angle(atoms[i][1:], atoms[j][1:], atoms[k][1:])
                angle_radians= np.radians(angle_degrees)
                # Calculate bending energy in degrees
                angle_energy_degrees = ka_degree * (angle_degrees - equilibrium_angle_degrees) ** 2
                bend_energy_degrees += angle_energy_degrees




                #angles energy in radians
                # Calculate bond angle in degrees
                angle_degrees = bond_angle(atoms[i][1:], atoms[j][1:], atoms[k][1:])

                # Calculate bending energy in degrees
                angle_energy_degrees = ka_degree * (angle_degrees - equilibrium_angle_degrees) ** 2
                bend_energy_degrees += angle_energy_degrees

                # Convert energy from degrees to radians
                conversion_factor = (np.pi / 180) ** 2
                angle_energy_radians = angle_energy_degrees * conversion_factor
                bend_energy_radians += angle_energy_radians

                # Increment angle count
                angle_count += 1




                # Print the results
                print(f"{atoms[i][0]}{i + 1} - {atoms[j][0]}{j + 1} - {atoms[k][0]}{k + 1}: "
                      f"{angle_degrees:>10.3f} {angle_radians:>10.3f} {angle_energy_degrees:>20.6f} {bend_energy_radians:>20.6f}")

# Print total bending energy
print(f"\nTotal bending energy in degrees: {bend_energy_degrees:.6f} kcal/mol")
print(f"Total bending energy in radians: {bend_energy_radians:.6f} kcal/mol")
print(f"Total number of angles calculated: {angle_count}")


C2 - C1 - H3:    110.805      1.934            59.599186             0.018155
C2 - C1 - H4:    113.234      1.976           488.084443             0.166834
C2 - C1 - H5:    105.807      1.847           477.313397             0.312232
H3 - C1 - H4:    111.697      1.949           168.995515             0.363711
H3 - C1 - H5:     95.349      1.664          7008.625162             2.498660
H4 - C1 - H5:    118.505      2.068          2837.923993             3.363141
C1 - C2 - H6:    113.910      1.988           680.587492             3.570460
C1 - C2 - H7:    108.124      1.887            66.234891             3.590636
C1 - C2 - H8:    109.544      1.912             0.067946             3.590657
H6 - C2 - H7:    110.905      1.936            69.124916             3.611714
H6 - C2 - H8:    108.830      1.899            15.708255             3.616499
H7 - C2 - H8:    105.160      1.835           659.229842             3.817312

Total bending energy in degrees: 25062.990078 kcal/mol
Total be

In [124]:

# Fixed bond length threshold for bonding (in Å)
bond_thresh = 1.6  # Threshold for bonding
torsion_params = {
    "v_n": 0.3,  # kcal/mol, torsional barrier
    "n": 3,      # Periodicity of torsion angle
}

torsion_n = []
# Function to calculate and print torsions
def calculate_and_print_torsions(atoms):
    coords = [atom[1:] for atom in atoms]  # Extract coordinates
    n_atoms = len(atoms)
    torsions = []
    torsions= torsion_n
    connectivity = [[] for _ in range(n_atoms)]

    # Build connectivity directly inside this function
    for i in range(n_atoms):
        for j in range(i + 1, n_atoms):
            if bond_length(coords[i], coords[j]) < bond_thresh:
                connectivity[i].append(j)
                connectivity[j].append(i)

    # Find all torsions directly
    for j in range(n_atoms):
        for a in range(len(connectivity[j])):
            k = connectivity[j][a]
            if k < j:
                continue
            for b in range(len(connectivity[j])):
                i = connectivity[j][b]
                if i == k:
                    continue
                for c in range(len(connectivity[k])):
                    l = connectivity[k][c]
                    if l == j or l == i:
                        continue
                    # Calculate dihedral angle
                    angle_deg, angle_rad = dihedral_angle(coords[i], coords[j], coords[k], coords[l])

                    # Calculate torsion energy in degrees and radians
                    torsion_energy_deg = torsion_params["v_n"] * (1 + np.cos(torsion_params["n"] * np.radians(angle_deg)))
                    torsion_energy_rad = torsion_params["v_n"] * (1 + np.cos(torsion_params["n"] * angle_rad))

                    # Append torsion data
                    torsions.append((i, j, k, l, angle_deg, angle_rad, torsion_energy_deg, torsion_energy_rad))

    # Print torsions and total energies
    torsion_energy_total_deg = 0.0
    torsion_energy_total_rad = 0.0
    print("\nList of all torsion angles with energies:")

    for (i, j, k, l, angle_deg, angle_rad, torsion_energy_deg, torsion_energy_rad) in torsions:
        torsion_energy_total_deg += torsion_energy_deg
        torsion_energy_total_rad += torsion_energy_rad

        print(f"{atoms[i][0]}{i + 1} - {atoms[j][0]}{j + 1} - {atoms[k][0]}{k + 1} - {atoms[l][0]}{l + 1}: "
              f"{angle_deg:>10.3f} {angle_rad: >10.3f} {torsion_energy_deg:>20.6f}")

    # Print total torsion energy
    print(f"\nTotal torsion energy (calculated using degrees): {torsion_energy_total_deg:.6f} kcal/mol")
    print(f"Total torsion energy (calculated using radians): {torsion_energy_total_rad:.6f} kcal/mol")
    print(f"Number of torsions calculated: {len(torsions)}")

# Example usage


calculate_and_print_torsions(atoms)


internal_coords = bond_count + angle_count +  len(torsion_n)
cartesian_coords = 3 * len(atoms)

print(f"Internal Coordinates: {internal_coords}")
print(f"Cartesian Coordinates: {cartesian_coords}")





List of all torsion angles with energies:
H3 - C1 - C2 - H6:     57.314      1.000             0.002961
H3 - C1 - C2 - H7:    -66.452     -1.160             0.016959
H3 - C1 - C2 - H8:    179.457      3.132             0.000121
H4 - C1 - C2 - H6:   -176.279     -3.077             0.005677
H4 - C1 - C2 - H7:     59.955      1.046             0.000001
H4 - C1 - C2 - H8:    -54.136     -0.945             0.014029
H5 - C1 - C2 - H6:    -44.879     -0.783             0.089212
H5 - C1 - C2 - H7:   -168.646     -2.943             0.051470
H5 - C1 - C2 - H8:     77.263      1.348             0.114432

Total torsion energy (calculated using degrees): 0.294863 kcal/mol
Total torsion energy (calculated using radians): 0.294863 kcal/mol
Number of torsions calculated: 9
Internal Coordinates: 28
Cartesian Coordinates: 24


In [123]:
Torsion

' '

In [113]:
# Function to calculate van der Waals energy and print as it iterates
def calculate_and_print_vdw_energy(atoms, bonds_new, angles_new):
    # Define epsilon and sigma values for Hydrogen and Carbon
    epsilon_values = {'H': 0.03, 'C': 0.07}  # in kcal/mol
    sigma_values = {'H': 1.20, 'C': 1.75}    # in Angstroms

    total_energy = 0.0

    print(f"{'Atom Pair':<15} {'Distance (Å)':>15} {'Energy (kcal/mol)':>20}")

    N = len(atoms)

    # Iterate over all unique atom pairs
    for i in range(N):
        for j in range(i + 1, N):
            calc_vdw = True  # Assume we will calculate vdW for this pair
            # Exclude bonded pairs
            for bond in bonds_new:
                if i in bond and j in bond:
                    calc_vdw = False
                    break
            # Exclude pairs involved in angles
            if calc_vdw:
                for angle in angles_new:
                    if i in angle and j in angle:
                        calc_vdw = False
                        break

            # Retrieve atom types and coordinates
            atom1_type, x1, y1, z1 = atoms[i]
            atom2_type, x2, y2, z2 = atoms[j]
            r_ij = math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)

            if calc_vdw:
                # Retrieve epsilon and sigma values
                epsilon_i = epsilon_values.get(atom1_type)
                epsilon_j = epsilon_values.get(atom2_type)
                sigma_i = sigma_values.get(atom1_type)
                sigma_j = sigma_values.get(atom2_type)

                if epsilon_i is None or epsilon_j is None:
                    raise ValueError(f"Unknown atom type: {atom1_type} or {atom2_type}")

                # Compute mixed epsilon and sigma using geometric mean
                epsilon_ij = math.sqrt(epsilon_i * epsilon_j)
                sigma_ij = 2*math.sqrt(sigma_i * sigma_j)

                # Compute Lennard-Jones potential
                term12 = (sigma_ij / r_ij) ** 12
                term6 = (sigma_ij / r_ij) ** 6
                energy_lj = 4 * epsilon_ij * (term12 - term6)

                # Accumulate the total energy
                total_energy += energy_lj
                energy = energy_lj
            else:
                # For uncalculated pairs, energy is zero
                energy = 0.0

            # Print the current pair details
            print(f"{atom1_type} {i + 1:<3}- {atom2_type} {j + 1:<3}: {r_ij:>15.4f} {energy:>20.4f}")

    print(f"\nTotal van der Waals Energy: {total_energy:.4f} kcal/mol")


# Example Input


# Calculate and print van der Waals energy
calculate_and_print_vdw_energy(atoms, bonds_new, angles_new)


Atom Pair          Distance (Å)    Energy (kcal/mol)
C 1  - C 2  :          1.5137               0.0000
C 1  - H 3  :          1.0647               0.0000
C 1  - H 4  :          1.0791               0.0000
C 1  - H 5  :          1.2236               0.0000
C 1  - H 6  :          2.1640               0.0000
C 1  - H 7  :          2.1325               0.0000
C 1  - H 8  :          2.1711               0.0000
C 2  - H 3  :          2.1377               0.0000
C 2  - H 4  :          2.1781               0.0000
C 2  - H 5  :          2.1903               0.0000
C 2  - H 6  :          1.0503               0.0000
C 2  - H 7  :          1.1033               0.0000
C 2  - H 8  :          1.1304               0.0000
H 3  - H 4  :          1.7741               0.0000
H 3  - H 5  :          1.6952               0.0000
H 3  - H 6  :          2.5002              -0.0204
H 3  - H 7  :          2.5003              -0.0204
H 3  - H 8  :          3.0658              -0.0213
H 4  - H 5  :          1.9804

# New Section

In [113]:
print_atom_coordinates(atoms)
print(f"\nTotal Stretching (Bond) Energy: {stretch_energy:.4f} kcal/mol")
print(f"Number of Bonds: {bond_count}")

print(f"\nTotal bending energy in degrees: {bend_energy_degrees:.6f} kcal/mol")
print(f"Total bending energy in radians: {bend_energy_radians:.6f} kcal/mol")
print(f"Total number of angles calculated: {angle_count}")
